This code, evaluates the best classifier found in the tuning phase agaisnt the rest of the dataset. 

In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.model_selection import StratifiedKFold
import time

data_path = os.path.join('.','tests','test_output')

In [ ]:
y_true = []

knn_y = []
svm_y = []
dt_y = []
# Run training and test on the rest of the dataset batch
# use best parameter found during hypertuning
for i in range(8):
    pca = PCA(n_components = 100)
    X = np.load(os.path.join(data_path,'x_split_{}.npy'.format(i)),mmap_mode='r')
    Y = np.load(os.path.join(data_path,'y_split_{}.npy'.format(i)),mmap_mode='r')
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, stratify = Y)
    
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    knn = KNeighborsClassifier(n_neighbors=1)
    svm = SVC(C=1000,kernel = 'linear')
    dt = tree.DecisionTreeClassifier()
    
    knn.fit(X_train,y_train)
    svm.fit(X_train,y_train)
    dt.fit(X_train,y_train)
    
    y_true.append(y_test)
    start_time = time.time()
    knn_y.append(knn.predict(X_test))
    print(time.time()-start_time)
    svm_y.append(svm.predict(X_test))
    print(time.time()-start_time)
    dt_y.append(dt.predict(X_test))
    print(time.time()-start_time)
    

Do q chochrans test to check if any of the 3 classifier are different.

In [ ]:
from mlxtend.evaluate import cochrans_q

for i in range(8):
    q, p_value = cochrans_q(y_true[i], 
                        knn_y[i], 
                        svm_y[i], 
                        dt_y[i])
    print("q:{},p:{}".format(q, p_value))

In [ ]:
for i in range(8):
    knn_sc = np.mean(y_true[i]==knn_y[i])
    print("knn score:{}".format(knn_sc))
    
    svm_sc = np.mean(y_true[i]==svm_y[i])
    print("svm score:{}".format(svm_sc))
    
    dt_sc = np.mean(y_true[i]==dt_y[i])
    print("dt score:{}".format(dt_sc))